# Build Map

## Map Show

In [10]:
import pandas as pd
from sodapy import Socrata
import webbrowser
import os
import numpy as np
import folium

client = Socrata("data.melbourne.vic.gov.au", None)


CPBL = client.get("vh2v-4nfs", limit=2000)  
#On-street Parking Bay Sensors:
#https://data.melbourne.vic.gov.au/Transport/On-street-Parking-Bay-Sensors/vh2v-4nfs

CPBL_df = pd.DataFrame.from_records(CPBL)

CPBR = client.get("ntht-5rk7", limit=20000)
#On-street Car Park Bay Restrictions:
#https://data.melbourne.vic.gov.au/Transport/On-street-Car-Park-Bay-Restrictions/ntht-5rk7
CPBR_df = pd.DataFrame.from_records(CPBR)


#*****
merge_df=pd.merge(CPBL_df,CPBR_df,left_on='bay_id',right_on='bayid', how='left')

df=merge_df.iloc[:,[0,1,2,3,4,5,9,10,11,12,13,16,17,18]]

d=dict(df['location'])
for i in np.arange(0,len(d)):
    del d[i]['human_address']
dictdf=pd.DataFrame([d])
df['location'] = pd.DataFrame(dictdf.values.T, index=dictdf.columns, columns=dictdf.index)


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
from map import mark_map
myMap = folium.Map(location=[-37.82, 144.95], zoom_start=14)

for i in range(len(df)):
        # Customize popup content
    test = folium.Html(
            '<b>Bay_id:{}</b></br> <b>Statues:{}</b></br> <b>Description:{}</b></br> <b>Duration:{}</b></br> <b>Disability{}</b></br> '.format(data.iloc[i]['bay_id'], data.iloc[i]['status'], data.iloc[i]['description1'],data.iloc[i]['duration1'],data.iloc[i]['disabilityext1']),
            script=True)
        
    popup = folium.Popup(test)
        
    if df.iloc[i]['status']=='Present':
        icon=folium.Icon(color='red', icon='info-sign')
    else:
        icon=folium.Icon(color='blue', icon='ok-sign')
        
    folium.Marker([df.iloc[i]['lat'], df.iloc[i]['lon']], popup=popup,icon=icon).add_to(myMap)
    myMap.add_child(folium.LatLngPopup())
myMap
# Save the map
#geo_name = 'Geo_Map.html'
#to save it in a file
#mark_map.save('images/'+geo_name)
#webbrowser.open('images/'+geo_name)

#image_path = "file://"+os.getcwd()+ '/'+ 'images/'+geo_name
#webbrowser.open(image_path)


## Map Filter

In [28]:
from geopy.distance import geodesic
from map import mark_map
import folium

def geofilter(data,pin,n):
    filter=pd.DataFrame(columns=data.columns.values)
    j=0
    for i in np.arange(0,data.shape[0]):
        d = geodesic(pin[0],data.iloc[i]['location'].values()).km
        if d<=n:
            filter.loc[j]= data.iloc[i]
            j=j+1
        else:
            continue
    filter.to_csv('Geofilter_Pin[{}]_Distance[{}].csv'.
                 format(pin, n))
    filter_map=mark_map(filter)
    folium.Marker(pin[0], popup='PIN', tooltip='PIN',icon=folium.Icon(color='black')).add_to(filter_map)
    # Save the map
    geo_name = 'filter_map_Pin[{}]_Distance[{}].html'. format(pin, n)
    #to save it in a file
    filter_map.save('images/'+geo_name)
    #webbrowser.open('images/'+geo_name)
    image_path = "file://"+os.getcwd()+ '/'+ 'images/'+geo_name
    webbrowser.open(image_path)
    return filter_map
    
data=df
pin=[(-37.8090, 144.9562)]
n=6.3
geofilter(data,pin,n)




AttributeError: 'NoneType' object has no attribute 'add_child'